# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [58]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

# Set style for plots
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [59]:
def extract_time_series_features(data, window_size):
    """Extract rolling window features from time series data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data (e.g., 'heart_rate', 'eda', etc.)
    window_size : int
        Size of the rolling window in seconds
    sampling_rate : int
        Sampling rate in Hz (samples per second)
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted features for each signal
    """
    # Your code here
    # 1. Calculate rolling window statistics
    # 2. Include mean, std, min, max, and autocorrelation
    
    # Calculate the number of samples in each window
    
    features = []

    n_windows = len(data) // window_size

    for i in range(n_windows):
        window_data = data.iloc[i*window_size:(i+1)*window_size]
        
        window_features = {
            'mean': window_data.mean(),
            'std': window_data.std(),
            'min': window_data.min(),
            'max': window_data.max(),
        }
        
        for column in window_data.columns:
            autocorr = window_data[column].autocorr(lag=1)
            window_features[f'autocorr_lag1_{column}'] = autocorr
        #autocorr = window_data.corr(window_data.shift(1)) # Adapted for autocorrelation at lag 1
        #window_features['autocorr_lag1'] = autocorr
        
        features.append(window_features)
    
    features_df = pd.DataFrame(features)
    
    # Return DataFrame with extracted features
    return features_df

# Separate run for each subject 
def process_subject_sessions(data, window_size=60):
    """Process the data for each subject and session, extracting time series features.
    
    Parameters
    ----------
    data : pd.DataFrame
        DataFrame containing the preprocessed physiological data
    window_size : int
        Size of the rolling window in seconds
    sampling_rate : int
        Sampling rate in Hz (samples per second)
        
    Returns
    -------
    pd.DataFrame
        DataFrame with time series features for each subject and session
    """
    all_features = []

    # Loop through each unique subject_id and session
    for subject_id in data['subject_id'].unique():
        mid_data = data[(data['subject_id'] == subject_id)]

        for session in mid_data['session'].unique():
            #print(f"Filtering data for subject {subject_id} and session {session}...")
            #session_data = data[(data['subject_id'] == subject_id) & (data['session'] == session)]
            #print(f"Number of rows after filtering: {len(session_data)}")
        
            #if session_data.empty:
            #    print(f"No data found for subject {subject_id} and session {session}")
            #    continue
            
            # Filter data for the specific current unique subject and session
            thr_data = data[(data['session'] == session)]
            session_data = thr_data.drop(columns=['subject_id', 'session']) # Temp drop data for next function
            
            # Apply the time series feature extraction function 
            session_features = extract_time_series_features(session_data, 
                                                            window_size)
            
            session_features['subject_id'] = subject_id # Re-add the data back
            session_features['session'] = session
            
            all_features.append(session_features) # Includes tracking data and subject and session data
    
    # Concatenate all the session-wise features into one DataFrame
    final_features_df = pd.concat(all_features, ignore_index=True)
    
    return all_features

In [60]:
# jupyter nbconvert --to script part1_exploration.ipynb

from part1_exploration import load_data

data = load_data(data_dir='data/raw')

In [61]:
session_data = data.drop(columns=['subject_id', 'session'])
session_features = extract_time_series_features(session_data, window_size=60)
print(session_data)

c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


                     timestamp  heart_rate       eda  temperature
418966 1970-01-01 00:00:12.510       88.92  0.014094        12.51
418967 1970-01-01 00:00:12.510       89.15  0.014094        12.51
418968 1970-01-01 00:00:12.510       89.38  0.014094        12.51
418969 1970-01-01 00:00:12.510       89.57  0.014094        12.51
418961 1970-01-01 00:00:12.550       88.00  0.015375        12.55
...                        ...         ...       ...          ...
144553 1970-01-01 00:00:35.790      172.18  0.125837        35.79
144572 1970-01-01 00:00:35.790      141.10  0.123275        35.79
144573 1970-01-01 00:00:35.790      138.80  0.127119        35.79
144574 1970-01-01 00:00:35.790      136.52  0.123275        35.79
144554 1970-01-01 00:00:35.790      171.32  0.129681        35.79

[442972 rows x 4 columns]


In [62]:
final_features_df = process_subject_sessions(data, window_size=60)

print(final_features_df)

c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\jelou\Documents\Datasci-223-Python-2\4-it-s-about-time-klouie07\.venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3045: RuntimeWarning: inva

[                                                   mean  \
0     timestamp      1970-01-01 00:00:12.604666666
h...   
1     timestamp      1970-01-01 00:00:12.719333333
h...   
2     timestamp      1970-01-01 00:00:12.764666666
h...   
3     timestamp      1970-01-01 00:00:12.836666666
h...   
4     timestamp      1970-01-01 00:00:13.028666666
h...   
...                                                 ...   
2010  timestamp      1970-01-01 00:00:32.984000
hear...   
2011  timestamp      1970-01-01 00:00:33.004000
hear...   
2012  timestamp      1970-01-01 00:00:33.112333333
h...   
2013  timestamp      1970-01-01 00:00:33.152833333
h...   
2014  timestamp      1970-01-01 00:00:33.196833333
h...   

                                                    std  \
0     timestamp      0 days 00:00:00.048554238
heart...   
1     timestamp      0 days 00:00:00.014481626
heart...   
2     timestamp      0 days 00:00:00.013712133
heart...   
3     timestamp      0 days 00:00:00.036576674
heart..

## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [63]:
def build_arima_model(series, order=(1,1,1), output_dir='plots'):
    """Fit an ARIMA model to the time series and generate diagnostic plots.
    
    Parameters
    ----------
    series : pd.Series
        Time series data to model
    order : tuple
        (p,d,q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots
        
    Returns
    -------
    statsmodels.tsa.arima.model.ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Your code here
    # 1. Fit ARIMA model
    # 2. Generate diagnostic plots:
    #    - Model fit plot
    #    - Residuals plot
    #    - Forecast plot
    # 3. Save plots to output directory

    # Fit the ARIMA model
    model = ARIMA(series, order=order)
    fitted_model = model.fit()

    # Generate fitted values and residuals
    fitted_values = fitted_model.fittedvalues
    residuals = fitted_model.resid

    # Actual vs Fitted Plot
    plt.figure(figsize=(10, 6))
    plt.plot(series, label="Actual", color='blue', alpha=0.6)
    plt.plot(fitted_model.fittedvalues, label="Fitted", color='red', alpha=0.6)
    plt.title("Actual vs Fitted")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.savefig(os.path.join(output_dir, 'actual_vs_fitted.png'))
    plt.close()

    # Plot residuals
    residuals = fitted_model.resid
    plt.figure(figsize=(10, 6))
    plt.subplot(211)
    plt.plot(residuals)
    plt.title("Residuals from ARIMA Model")
    plt.savefig(os.path.join(output_dir, 'residuals_plot.png'))
    plt.close()
    
    # Plot histogram of residuals
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.title("Histogram of Residuals")
    plt.savefig(os.path.join(output_dir, 'histogram_residuals.png'))
    plt.close()

    print(f"Saved ARIMA plots")

    return fitted_model

In [ ]:
fitted_model = build_arima_model(data, order=(1, 1, 1), output_dir='plots')

#print(fitted_model.summary())